In [1]:
import pymysql
from config.DatabaseConfig import * # DB 접속 정보 불러오기

DB_HOST = "localhost"
DB_USER = "myuser118"
DB_PASSWORD = "1234"
DB_NAME = "mydb118"

def DatabaseConfig():
    global DB_HOST, DB_USER, DB_PASSWORD, DB_NAME

db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 테이블 생성 sql 정의
    sql = '''
      CREATE TABLE IF NOT EXISTS `product_cafe` (
      `id` INT UNSIGNED NOT NULL AUTO_INCREMENT,
      `name` VARCHAR(45) NULL,
      `price` INT NOT NULL,
      `detail` VARCHAR(500) NULL,
      `image` VARCHAR(200) NULL,
      `latte` INT NOT NULL,
      `coffee` INT NOT NULL,
      `drink` INT NOT NULL,
      `food` INT NOT NULL,
      `recommend` INT NOT NULL,
      PRIMARY KEY (`id`))
    ENGINE = InnoDB DEFAULT CHARSET=utf8
    '''

    # 테이블 생성
    with db.cursor() as cursor:
        cursor.execute(sql)

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()



In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('product.xlsx')
df

,name,price,detail,image,Latte,coffee,drink,food
0,갈릭 치즈 브레드,4600,풍부한 갈릭버터스프레드와 조화를 이룬 진한 체다치즈에 바삭한 갈릭칩을 얹어 촉촉한 ...,IMG_1566780567234.png,0,0,0,1
1,고구마 라떼,4000,호박고구마를 활용하여 달콤하고 고소한 고구마의 풍미가 진하게 느껴지는 음료,IMG_1647322145592.png,1,0,0,0
2,고구마 말랭이,3000,어떤 첨가물도 사용하지 않은 100% 고구마 그대로의 맛,IMG_1535034350410.png,0,0,0,1
3,골드키위주스,3600,고급 골드키위를 사용하여 비타민 C가 풍부하고 상큼한 음료,IMG_1647324243707.png,0,0,1,0
4,그린루이보스,3600,복숭아 한 조각을 베어 문 잔향이 입안에 남는 맑은 수색의 루이보스 티,IMG_1647324593188.png,0,0,1,0
...,...,...,...,...,...,...,...,...
180,화이트 초콜릿,2300,화이트 초콜렛 위에 곁들여진 휘핑크림의 달콤함과 부드러움이 단연 돋보이는 음료,IMG_1647321721558.png,0,0,1,0
181,화이트 초콜릿 모카,4500,화이트 초콜렛과 에스프레소의 조화로운 만남에 달콤한 휘핑크림까지 함께 즐길 수 있는 음료,IMG_1647321357192.png,0,1,0,0
182,화이트초코 마카다미아 쿠키,2000,마카다미아 넛의 고소함과 화이트초콜렛의 달콤함이 조화를 이루는 쿠키. 이디야만의 반...,IMG_1641946096782.png,0,0,0,1
183,흑당 라떼,4300,진하고 달콤한 흑당과 고소한 우유가 어우러진 논커피 음료,IMG_1647322029671.png,1,0,0,0


In [3]:
import pymysql
import openpyxl

from config.DatabaseConfig import * # DB 접속 정보 불러오기


In [4]:
# 학습 데이터 초기화
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
            delete from product_cafe
        '''
    with db.cursor() as cursor:
        cursor.execute(sql)

    # auto increment 초기화
    sql = '''
    ALTER TABLE product_cafe AUTO_INCREMENT=1
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)


# db에 데이터 저장
def insert_data(db, xls_row):
    name, price, detail, image, latte, coffee, drink, food = xls_row

    sql = '''
        INSERT product_cafe(name, price, detail, image, latte, coffee, drink, food, recommend) 
        values(
         '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (name.value, price.value, detail.value, image.value, latte.value, coffee.value, drink.value, food.value, recommend.value)

    # 엑셀에서 불러온 cell에 데이터가 없는 경우, null 로 치환
    sql = sql.replace("'None'", "null")

    with db.cursor() as cursor:
        cursor.execute(sql)
        db.commit()


In [5]:
train_file = r'product.xlsx'
db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 기존 학습 데이터 초기화
    all_clear_train_data(db)

    # 학습 엑셀 파일 불러오기
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['product']
    for row in sheet.iter_rows(min_row=2): # 해더는 불러오지 않음
        # 데이터 저장
        insert_data(db,row)

    wb.close()

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()



In [1]:
from config.DatabaseConfig import *
from utils.Database import Database
from utils.FindProduct import FindProduct
import pymysql
import openpyxl
from functools import reduce


db = Database(
        host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db_name=DB_NAME
    )

db.connect()
answer = db.select_all("select * from product_cafe where latte = 1")

In [2]:
detail = reduce(lambda r,e : r.append(e['detail']) or r, answer, [])
detail

['호박고구마를 활용하여 달콤하고 고소한 고구마의 풍미가 진하게 느껴지는 음료',
 '향긋한 녹차의 쌉쌀한 맛에 우유가 더해져 부담없이 즐길 수 있는 음료',
 '토피넛 파우더와 마카다미아향이 어우러져 고소하고 달콤한 음료와 아몬드 브리틀의 바삭한 식감을 즐길 수 있는 음료',
 '디카페인 콜드브루의 깔끔하고 쌉싸름한 풍미가 고소한 우유와 만나 누구나 부담없이 즐길 수 있는 음료 ',
 '부드러운 라떼에 은은한 바닐라 향이 더해져 더욱 달콤하게 즐길 수 있는 음료',
 '진하고 달콤한 흑당에 쫄깃한 버블이 어우러진 논커피 음료',
 '이디야의 노하우가 집약된 골드 블렌드 커피믹스에 쫀득한 버블을 더해 엑스트라 사이즈로 즐길 수 있는 음료',
 '베트남풍 연유의 달콤한 맛과 에스프레소가 절묘하게 어우러져 누구나 쉽게 즐길 수 있는 부드러운 커피 음료',
 '몸에 이로운 곡물이 들어가 든든하고 포만감을 주는 메뉴, 한끼 식사 대용으로도 충분한 음료',
 '진한 에스프레소와 부드러운 우유가 만나 고소한 풍미를 더하는 메뉴 가장 대중적인 메뉴',
 '콜드브루의 깔끔하고 쌉싸름한 풍미가 고소한 우유와 만나 누구나 부담없이 즐길 수 있는 음료',
 '부드러운 크림과 에스프레소, 너티 플레이버가 어우러져 만들어진 완벽한 한 잔의 라떼',
 '다양한 견과류의 맛과 달콤한 코코아가 조화롭게 어우러진 달콤한 음료',
 '진하고 달콤한 흑당과 고소한 우유가 어우러진 논커피 음료']

In [3]:
answer

[{'id': 2,
  'name': '고구마 라떼',
  'price': 4000,
  'detail': '호박고구마를 활용하여 달콤하고 고소한 고구마의 풍미가 진하게 느껴지는 음료',
  'image': 'IMG_1647322145592.png',
  'latte': 1,
  'coffee': 0,
  'drink': 0,
  'food': 0},
 {'id': 11,
  'name': '녹차 라떼',
  'price': 3700,
  'detail': '향긋한 녹차의 쌉쌀한 맛에 우유가 더해져 부담없이 즐길 수 있는 음료',
  'image': 'IMG_1647321791414.png',
  'latte': 1,
  'coffee': 0,
  'drink': 0,
  'food': 0},
 {'id': 16,
  'name': '더블 토피넛 라떼',
  'price': 5200,
  'detail': '토피넛 파우더와 마카다미아향이 어우러져 고소하고 달콤한 음료와 아몬드 브리틀의 바삭한 식감을 즐길 수 있는 음료',
  'image': 'IMG_1647319791824.png',
  'latte': 1,
  'coffee': 0,
  'drink': 0,
  'food': 0},
 {'id': 23,
  'name': '디카페인 콜드브루 라떼',
  'price': 4200,
  'detail': '디카페인 콜드브루의 깔끔하고 쌉싸름한 풍미가 고소한 우유와 만나 누구나 부담없이 즐길 수 있는 음료 ',
  'image': 'IMG_1647320357663.png',
  'latte': 1,
  'coffee': 0,
  'drink': 0,
  'food': 0},
 {'id': 56,
  'name': '바닐라 라떼',
  'price': 3900,
  'detail': '부드러운 라떼에 은은한 바닐라 향이 더해져 더욱 달콤하게 즐길 수 있는 음료',
  'image': 'IMG_1647321057747.png',
  'latte': 1,
  'coff